<h1 style="font-family:Impact,Arial;font-size:50px">Load SQL</h1>
<p> This code loads the raw data from the VAST database. It uses a PSQL connection to pull the raw tables and columns, it then joins and converts the tables to usable format in Pandas in conjunction with the scripts in git. 

In [13]:
import math
import pandas as pd
import numpy as np
import psycopg2 as pg
import pandas.io.sql as psql

In [14]:
# get connected to the database
connection = pg.connect("dbname=postgres user=postgres password=sqg4179eq")

In [15]:
%%time
image_table = psql.read_sql("SELECT id, rms, time, jd, cube_id, ra, dec FROM vast_image", connection)
image_table.columns = (['image_id', 'rms', 'time', 'jd', 'cube_id', 'im_ra', 'im_dec'])

CPU times: user 63.6 ms, sys: 24.4 ms, total: 87.9 ms
Wall time: 733 ms


In [16]:
%%time
dataframe = psql.read_sql('SELECT id, source_id, image_id, good_fit, ra, err_ra, dec, err_dec, bmaj, bmin, pa, raw_total_flux, err_raw_total_flux, raw_peak_flux, err_raw_peak_flux FROM vast_flux', connection)

CPU times: user 2min 54s, sys: 5min 13s, total: 8min 8s
Wall time: 17min 8s


In [17]:
%%time
raw_data = (pd.merge(dataframe, image_table, left_on='image_id', right_on='image_id'))

CPU times: user 13.8 s, sys: 8.4 s, total: 22.2 s
Wall time: 24.3 s


In [18]:
def vectorized_distance_on_unit_sphere(df):
    degrees_to_radians = np.pi/180.0
    phi1 = df.ra*degrees_to_radians
    phi2 = df.im_ra*degrees_to_radians

    theta1 = df.dec*degrees_to_radians
    theta2 = df.im_dec*degrees_to_radians
    
    cosine = (np.cos(phi1)*np.cos(phi2)*np.cos(theta1 - theta2) +
           np.sin(phi1)*np.sin(phi2))
    dist_from_centre = np.arccos(cosine)
    return (dist_from_centre/3.142)*180

## Conversions

In [19]:
raw_data['raw_peak_flux'] = raw_data['raw_peak_flux']*(1.0/1000.0) # Conversion to Jy
raw_data['err_raw_peak_flux'] = raw_data['err_raw_peak_flux']*(1.0/1000.0) # Conversion to Jy
raw_data['datetime'] = pd.to_datetime(raw_data.time, utc=True)

## Calculate the distance between the source and the image centre ( then create column)

In [20]:
%%time
raw_data['distance'] = vectorized_distance_on_unit_sphere(raw_data)

CPU times: user 1.84 s, sys: 2.82 s, total: 4.66 s
Wall time: 3.33 s


## Add in the gains to the data file

In [21]:
# SORT OUT LATER
#%%time
#gains = pd.read_table('../all_gains.txt', ',')
#raw_data = pd.merge(raw_data, gains, left_on='Image', right_on='Image')

## Filter good fits only

In [22]:
raw_data = raw_data[raw_data.good_fit == True]

## Save the reduced data file

In [26]:
%%time
# Parq moaned about my datetime column - ask Adel
raw_data= raw_data.reset_index()
raw_data.to_feather('mwats_raw_data_Feb_SQL.fth')

CPU times: user 7.22 s, sys: 13.9 s, total: 21.1 s
Wall time: 19.2 s
